In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

## 1.1 生成数据集 （修改window_len即可）

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('train.csv',header = None)

# data = data.to_numpy()
total_num = data.shape[0]
total_len = data.shape[1]
total_num
total_len

output_size = 56

111

735

In [4]:
# max_min 归一化
def max_min(x,total_num):
    min_val = x.values.min(axis=1)
    max_val = x.values.max(axis=1)
    values = (x.values-min_val.reshape(total_num,1))/(max_val - min_val).reshape(total_num,1)
    return values,min_val,max_val

In [5]:
# 最后生成测试数据时进行还原，然后与测试的label进行比较
def max_min_reverse(x_numpy,min_val,max_val):
    values = (x_numpy + min_val.reshape(total_num,1)) * (max_val - min_val).reshape(total_num,1)
    return values

In [6]:

# max_min 归一化
data,min_val,max_val = max_min(data,total_num)

# data = max_min_reverse(data,min_val,max_val)


In [7]:
data[0,:]

array([0.17652547, 0.19387946, 0.27075947, 0.45698824, 0.35062512,
       0.21869752, 0.20171674, 0.15282702, 0.26180258, 0.31293152,
       0.44803135, 0.44485912, 0.24034335, 0.25527151, 0.22728121,
       0.31349132, 0.47564844, 0.44243329, 0.42955775, 0.09852584,
       0.18950908, 0.13342041, 0.2358649 , 0.29128569, 0.43758164,
       0.43571562, 0.26348199, 0.22560179, 0.18660198, 0.24706102,
       0.26030976, 0.46725135, 0.43814144, 0.22261616, 0.1225975 ,
       0.04217205, 0.29427132, 0.23885053, 0.32300802, 0.09740623,
       0.15898489, 0.16626236, 0.15431984, 0.29837656, 0.29632394,
       0.45101698, 0.42974436, 0.15898489, 0.64582945, 0.21888412,
       0.26982646, 0.3021086 , 0.43161038, 0.09106177, 0.15898489,
       0.45381601, 0.1977981 , 0.25639112, 0.29688375, 0.46575854,
       0.37339056, 0.24818063, 0.20190334, 0.17465945, 0.2362381 ,
       0.30901288, 0.1599179 , 0.27729054, 0.20899422, 0.22560179,
       0.23325247, 0.27075947, 0.3493189 , 0.44915096, 0.37824

In [8]:
# 划分训练集和验证集

window_len = 56

train_series = data[:,0:total_len-output_size]
train_size = train_series.shape[1]

y_val_list = [data[i,total_len-output_size:] for i in range(total_num)]
y_val_list[0].shape

x_test_list = [data[i,total_len-window_len:] for i in range(total_num)]
x_test_list[0].shape

(56,)

(56,)

In [9]:
# todo 滑动窗口截取
# 每个样本的shape为(111,window_len) (111,window_len:window_len+56)


x_train_list = []
y_train_list = []
x_val_list = []
for i in range(total_num):
    for j in range(0,train_size - window_len-output_size,20):
        x_train = train_series[i,j:j+window_len]
        y_train = train_series[i,j+window_len:j+window_len+output_size]
        x_train_list.append(x_train)
        y_train_list.append(y_train)


x_val_list = [train_series[i,train_size-output_size-window_len:] for i in range(total_num)]

dataset ={
    'x_train':x_train_list,
    'y_train':y_train_list,
    'x_val':x_val_list,
    'y_val':y_val_list,
    'x_test':x_test_list,
    'max_val':max_val,
    'min_val':min_val
}

# todo 问题在于 700多维的数据，LSTM记不住该如何划分。

In [10]:
import pickle
with open('dataset_%d.pkl'%window_len,'wb') as f:
    pickle.dump(dataset,f)

## 1.2 封装数据

In [11]:
import torch
from torch import nn
import torch.functional as f
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim

In [12]:
class MyDataset(Dataset):
    def __init__(self,x_train,y_train):
        super(Dataset,self).__init__()
        self.length = len(x_train)
        self.data = [torch.Tensor(x_train[i]) for i in range(self.length)]
        self.label = [torch.Tensor(y_train[i]) for i in range(self.length)]
        
    
    def __getitem__(self,index):
        return self.data[index],self.label[index]
    
    def __len__(self):
        return self.length

In [13]:
# 封装训练和验证集
x_train = dataset['x_train']
y_train = dataset['y_train']
x_val = dataset['x_val']
y_val = dataset['y_val']
min_val = dataset['min_val']
max_val = dataset['max_val']
    

train_dataset = MyDataset(x_train,y_train)
val_dataset = MyDataset(x_val,y_val)

train_loader = DataLoader(train_dataset,batch_size = 32,shuffle = True)
val_loader = DataLoader(val_dataset,batch_size = total_num)


In [14]:
# 封装测试集
x_test = dataset['x_test']
y_test = pd.read_csv('test.csv',header = None)
y_test = [y_test.iloc[i,:].to_numpy() for i in range(len(x_test))]
test_dataset = MyDataset(x_test,y_test)
test_loader = DataLoader(test_dataset,batch_size = total_num)

# 3.搭建模型

In [15]:
import logging
logger = logging.getLogger(__name__)
logger.setLevel(level = logging.INFO)
handler = logging.FileHandler("log.txt")
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

In [16]:
from enum import Enum
class LossFunction(Enum):
    MAE = 1
    MSE = 2
    SmoothL1 = 3

class Mode(Enum):
    Trian = 1
    Valid = 2
    Test = 3

class Optim(Enum):
    SGD = 1
    Adam = 2

In [17]:
def initial_weights(model):
    for m in model.modules():
#         if isinstance(m, nn.LSTM):
#             [nn.init.orthogonal_(para) for name,para in m.name_parameters() if 'weight' in name]
                
        if isinstance(m,nn.Linear):
            nn.init.kaiming_normal_(m.weight.data)
    

class Encoder(nn.Module):
    def __init__(self,input_size,rnn_inpt_size,rnn_hidden_size,dropout_rate=0.2):
        super(Encoder,self).__init__()
        self.input_size = input_size
        self.rnn_input_size = rnn_inpt_size
        self.rnn_hidden_size = rnn_hidden_size
        
        self.linear = nn.Linear(input_size,rnn_inpt_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.lstm = nn.LSTM(rnn_inpt_size,rnn_hidden_size,batch_first=True)
        self.apply(initial_weights)
    
    def forward(self,x):
        emb = self.linear(x) # batch size * seq_len
        _,(h_n,_) = self.lstm(emb.reshape(x.shape[0],1,self.rnn_input_size))
        
        return h_n
        
    

In [18]:

class Decoder(nn.Module):
    def __init__(self,ouput_size,rnn_inpt_size,rnn_hidden_size,dropout_rate=0.2):
        super(Decoder,self).__init__()
        self.output_size = output_size
        self.rnn_input_size = rnn_inpt_size
        self.rnn_hidden_size = rnn_hidden_size
        
        self.linear = nn.Linear(rnn_hidden_size,ouput_size)
#         self.dropout = nn.Dropout(dropout_rate)
#         self.lstm = nn.LSTM(rnn_inpt_size,rnn_hidden_size,batch_first=True)
#         self.apply(initial_weights)
    
    def forward(self,x):
        emb = self.linear(x.reshape(-1,self.rnn_hidden_size)) # batch size * seq_len
        
        return emb
        

In [19]:
class Seq2Seq(nn.Module):
    def __init__(self,input_size,ouput_size,rnn_inpt_size,rnn_hidden_size,loss_type=LossFunction.MAE,dropout_rate=0.2):
        super(Seq2Seq,self).__init__()
        self.name = 'LSTM-based seq-to-seq'
        self.loss_type = loss_type
        self.encoder = Encoder(input_size,rnn_inpt_size,rnn_hidden_size,dropout_rate)
        self.decoder = Decoder(ouput_size,rnn_hidden_size,rnn_hidden_size,dropout_rate)
#         self.min = min_val
#         self.max = max_val
        if loss_type == LossFunction.MAE:
            self.loss_fun = nn.L1Loss()
        elif loss_type == LossFunction.MSE:
            self.loss_fun = nn.MSELoss()
        elif loss_type == LossFunction.SmoothL1:
            self.loss_fun = nn.SmoothL1Loss()
        else:
            raise ValueError("please check loss_fun type!")
        
    def forward(self,x,label=None):
        out = self.encoder(x)
        out = self.decoder(out)
#         if mode == Mode.Test:
#             out = max_min_reverse(out.cpu().detach().numpy(),self.min,self.max)
        if label is not None:
            loss = self.loss_fun(out,label)
            out = (out,loss)
            
        return  out

    

# 4.训练模型

In [20]:
from matplotlib import pyplot as plt

In [21]:
def smape(y_true, y_pred):
    return 2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100

In [22]:

class Trainer:
    def __init__(self,model,train_loader,val_loader,test_loader,min_val,max_val,train_args):
        super(Trainer,self).__init__()
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.lr = train_args['lr']
        self.batch_size = train_args['batch_size']
        self.optim_type = train_args['optim']
        self.epochs = train_args['epochs']
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu' 
        self.model.to(self.device)
        self.min_loss = 1e8
        self.best_model = None
        self.normalize_min_val = min_val
        self.mormalize_max_val = max_val
        if self.optim_type == Optim.SGD:
            self.optim = optim.SGD(self.model.parameters(),lr = self.lr)
        elif self.optim_type == Optim.Adam:
            self.optim = optim.Adam(self.model.parameters(),lr = self.lr)
        else:
            raise ValueError("Not a recognized optimizer")
    
    def __train_one_peoch(self):
        processbar = tqdm(range(self.train_loader.__len__()))
        self.model.train()
        train_loss = []
        train_smape = []
        for idx,(x_batch,y_batch) in enumerate(train_loader):
            x_batch = x_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            pred,loss = self.model(x_batch,y_batch)
            self.optim.zero_grad()
            loss.backward()
            self.optim.step()
            train_loss.append(loss.item())
            train_smape.append(SMAPE(pred.reshape(-1),y_batch))
            processbar.update(1)
        
        self.model.eval()
        with torch.no_grad():
            val_loss = []
            val_smape = []
            for idx,(x_batch,y_batch) in enumerate(val_loader):
                x_batch = x_batch.to(self.device)
                y_batch = y_batch.to(self.device)
                pred,loss = self.model(x_batch,y_batch)
                val_loss.append(loss.item())
                val_smape.append(self.SMAPE(pred.reshape(-1),y_batch))
            epoch_train_loss = sum(train_loss) / len(train_loss)
            epoch_train_smape = sum(train_smape) / len(train_smape)
            epoch_val_loss = sum(val_loss) / len(val_loss)
            epoch_val_smape = sum(val_smape) / len(val_smape)
            
        if epoch_val_loss < self.min_loss:
            self.min_loss = epoch_val_loss
            self.best_model = self.model

        return epoch_train_loss,epoch_val_loss,epoch_train_smape,epoch_val_smape
    
    
    def train(self):
        train_loss_list = []
        val_loss_list = []
        epoch_train_smape_list = []
        epoch_val_smape_list = []
        for i in range(self.epochs):
            train_loss,val_loss,epoch_train_smape,epoch_val_smape = self.__train_one_peoch()
            info_str = "train_loss:%.4f val_loss:%.4f train_smape:%.4f val_smape:%.4f\n" % (train_loss,val_loss,epoch_train_smape,epoch_val_smape)
            print(infor_str)
            train_loss_list.append(train_loss)
            val_loss_list.append(val_loss)
            epoch_train_smape_list.append(epoch_train_smape)
            epoch_val_smape_list.append(epoch_val_smape)
            if i % 21 == 0:
                self.save_model(i)
        title = 'len=%d + %s + %s + %s + '%(window_len,self.best_model.name,self.best_model.loss_type,self.optim_type.name)
        info_str = title + 'train_loss:%.4f val_loss:%.4f train_smape:%.4f val_smape:%.4f\n'%(train_loss,val_loss,epoch_train_smape,epoch_val_smape)
        logging.info(info_str)
        draw(train_loss_list,val_loss_list,'epochs','loss',['train_loss','val_loss'],title)
        draw(epoch_train_smape_list,epoch_val_smape_list,'epochs','smape',['train_smape','val_smape'],title)
        
        
    def draw(self,y1,y2,xlabel,ylabel,lengend,title):
        x = [i for i in range(len(y1))]
        plt.plot(x,y1,x,y2)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.lengend(lengend)
        plt.title(title)
        plt.savefig("%s.png"%title)
        plt.show()
        
        
    def eval(self,test_loader):
        self.model.eval()
        test_loss = []
        with torch.no_grad():
            # batch_size == total_num
            for idx,(x_batch,y_batch) in enumerate(test_loader):
                x_batch = x_batch.to(self.device)
                y_batch = y_batch.to(self.device)
                pred,loss = self.model(x_batch,y_label)
                test_loss.append(loss.item())
            test_loss = sum(test_loss) / len(test_loss)
            smape_val = self.SMAPE(pred.reshape(-1),y_batch,Mode.Test)
            
        return test_loss,smape_val

    
    def SMAPE(self,pred,y_batch,mode=Mode.Trian):
        pred = max_min_reverse(pred.cpu().numpy(),self.normalize_min_val,self.nmormalize_max_val)
        if mode == Mode.Test:
            y_batch = y_batch.cpu().numpy().reshape(-1)
        else:
            y_batch = max_min_reverse(y_batch.cpu().numpy(),self.normalize_min_val,self.nmormalize_max_val)
        smape_val= smape(pred.reshape(-1),y_batch)
        return smape_val
        
    def save_model(self,epoch=None):
        if epoch is not None:
            torch.save(self.best_model,'./checkpoint/len=%d + %s + %s + %s + checkpoint: %d.pt'
                       %(window_len,self.best_model.name,self.best_model.loss_type,self.optim_type.name,epoch))
        torch.save(self.best_model,'./checkpoint/len=%d + %s + %s + %s + test.pt'
                       %(window_len,self.best_model.name,self.best_model.loss_type,self.optim_type.name))

In [23]:
model_args = {
    "input_size": window_len+output_size,
    'output_size':output_size,
    "rnn_inpt_size" : 128,
    "rnn_hidden_size": 256,
    "loss_type":LossFunction.MAE,
    'dropout_rate':0.2
}

train_args = {
    'lr' : 3e-4,
    'batch_size' : 32,
    'dropout_rate' : 0.2,
    'optim' : Optim.SGD,
    'epochs' : 300,
}

model = Seq2Seq(input_size=model_args["input_size"],ouput_size=model_args['output_size'],
                rnn_inpt_size=model_args['rnn_inpt_size'],rnn_hidden_size=model_args['rnn_hidden_size'],
                loss_type=model_args['loss_type'],dropout_rate=model_args['dropout_rate'])

trainer = Trainer(model,train_loader,val_loader,test_loader,min_val,max_val,train_args)



In [24]:
from tqdm import tqdm
trainer.train()

  0%|                                                                                          | 0/101 [00:00<?, ?it/s]

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x56 and 112x128)